In [76]:
"""
Experiments with paper,
Explainable Verbal Deception Detection using TransformersLoukas Ilias1, Felix Soldner2,3, and Bennett Kleinberg3,4
"""
import openai
import os

openai.api_key = os.environ["OPENAI_API_KEY"]  # source the ~/.zshrc file

# https://platform.openai.com/docs/guides/rate-limits/error-mitigation
from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential,
)  # for exponential backoff

# constants, until you change them ;-)
new_line = '\n'
nb_test_samples = 10
nb_few_shot_samples_of_each_class = 5 # truth and deception
delimiter = '```\n'
MODEL = "gpt-3.5-turbo"
# MODEL = "text-davinci-003"
# MODEL = "gpt-4"


In [67]:
import pandas as pd
df = pd.read_csv ('sign_events_data_statements.csv')
# simple EDA
# print(df)
# print(df.columns)
print(f'shape: {df.shape}')  # should be 1640 x 6


shape: (1640, 6)


In [68]:
def filter_by_class(df, category):
   return df[df['outcome_class']== category]

truth_df = filter_by_class(df, 't')
# print(truth_df)
print(f'truth df shape: {truth_df.shape}')  # should be 1640 x 6

# replace with a more expressive word, truthful
truth_df['outcome_class'] = df['outcome_class'].replace('t','truthful')
print(truth_df)

deceit_df = filter_by_class(df, 'd')
# print(deceit_df)
print(f'deceit df shape: {deceit_df.shape}')  # should be 1640 x 6

# replace with a more expressive word, deceitful
deceit_df['outcome_class'] = df['outcome_class'].replace('d','deceitful')
print(deceit_df)

# pick random non-repeating rows
def randon_non_repeating(df, quantity):
    import random
    rand_df = pd.DataFrame()
    random_list = random.sample(range(df.shape[0]), quantity)
    print("non-repeating random numbers are:")
    return df.iloc[random_list], random_list

random_truth_df, truth_indices_list = randon_non_repeating(truth_df, nb_few_shot_samples_of_each_class)
print(f'random truth list:\n, {random_truth_df}')
print(f'truth indices:" {truth_indices_list}')

random_deceit_df, deceit_indices_list = randon_non_repeating(deceit_df, nb_few_shot_samples_of_each_class)
print(f'random deceit list:\n, {random_deceit_df}')
deceit_indices_list = [x + truth_df.shape[0] for x in deceit_indices_list] # do this to exclude from poriginal list
print(f'deceit indices: {deceit_indices_list}')

random_truth_deceit_df = pd.concat([random_truth_df, random_deceit_df])
few_shot_list = truth_indices_list + deceit_indices_list
print(f'truth + deceit indices" {few_shot_list}')


truth df shape: (783, 6)
                                             signevent   
0                                  My brothers wedding  \
1                   Going to collect 2 new pet rabbits   
2    Getting dinner with my friend Shan and my boyf...   
3    mountain bike ride with my boyfriend and daughter   
4    I will be going to the cat cafe in Glasgow on ...   
..                                                 ...   
778                          Go to NJ to visit cousins   
779  I am going to visit my mum and gran who I've n...   
780                                           Swimming   
781                    Going for coffee with the girls   
782                    Going to the cinema with Yasmin   

                                                    q1   
0    My little brother is getting married next Satu...  \
1    I will be driving for 80-90 minutes across Lon...   
2    We were planning to get dinner somewhere near ...   
3    We are going to cannock chase with the mo

/tmp/ipykernel_9838/3924573259.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  truth_df['outcome_class'] = df['outcome_class'].replace('t','truthful')
/tmp/ipykernel_9838/3924573259.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  deceit_df['outcome_class'] = df['outcome_class'].replace('d','deceitful')


#### Setup the OpenAI call

In [69]:
def get_chat_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(6))
def get_chat_completion_with_backoff(prompt, model="gpt-3.5-turbe"):
    return get_chat_completion(prompt, model)

def get_completion(prompt, model='text-davinci-003'):
    response = openai.Completion.create(
        prompt=prompt,
        model=model
    )
    return response.choices[0]['text']

In [70]:
intro = f"""
You are tasked to judge whether the response to questions are truthful or deceitful.
You'll be presented with three pieces of information:
(1) The title of the activity.
(2) Response to question #1: “Please describe your activity. Be as specific as possible.”
(3) Response to question #2: “What information can you give us to reassure us that you are telling the truth”
Complete the response with either 'truthful' or 'deceitful'
Here are a few examples delimited by triple backticks:
"""

response_1_heading = """Response #1:\n"""
response_2_heading = """Response #2:\n"""

def construct_activity_scenario(row):
    # activity_header = 'Title of the Activity: ' + new_line
    activity_header = 'Activity: ' + new_line
    activity_description_header = 'Question #1: \nPlease describe your activity. Be as specific as possible.'

    activity_reassurance_header = 'Question #2: \nWhat information can you give us to reassure us that you are telling the truth.'

    activity = activity_header + row['signevent'] + new_line
    q1 = activity_description_header + new_line + response_1_heading + row['q1'] + new_line
    q2 = activity_reassurance_header + new_line + response_2_heading + row['q2'] + new_line
    return activity + q1 + q2

def construct_outcome(row):
    outcome = "Your judgement on the responses are truthful or deceitful:\n"
    return outcome  + row['outcome_class'] + new_line * 2
    

def construct_few_shot_prompt(few_shot_df, infer_row):
    # constructed as a list
    prompt = []
    prompt.append(intro)

    
    for _, row in few_shot_df.iterrows():
        prompt.append(delimiter)
        prompt.append(construct_activity_scenario(row))
        prompt.append(construct_outcome(row))
        prompt.append(delimiter)
    

    prompt.append(delimiter)
    prompt.append(construct_activity_scenario(infer_row))
    prompt.append(construct_outcome(infer_row)) # has to have a blank outcome to be filled by the llm
    prompt.append(delimiter)

    return prompt

def create_test_indices(df, total, exclude_list):
    import random
    rand_list = []
    count = 0
    while count < total:
        rand_row = random.randrange(df.shape[0])
        if rand_row not in exclude_list:
            rand_list.append(rand_row)
            count += 1
    return rand_list





In [74]:
test_indices = create_test_indices(df, nb_test_samples, few_shot_list)  # exclude the ones in the few shot list
# test_indices = [1435]
print(f'test indices: {test_indices}')



test indices: [1269, 1214, 15, 435, 1278, 1260, 1621, 796, 339, 73]


/tmp/ipykernel_9838/1997141771.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  infer_row['outcome_class'] = ''


INDEX: 1269 GROUND TRUTH: deceitful, RESPONSE: truthful - wrong
INDEX: 1214 GROUND TRUTH: deceitful, RESPONSE: deceitful - correct
INDEX: 15 GROUND TRUTH: truthful, RESPONSE: truthful - correct
INDEX: 435 GROUND TRUTH: truthful, RESPONSE: deceitful - wrong
INDEX: 1278 GROUND TRUTH: deceitful, RESPONSE: truthful - wrong
INDEX: 1260 GROUND TRUTH: deceitful, RESPONSE: deceitful - correct
INDEX: 1621 GROUND TRUTH: deceitful, RESPONSE: deceitful - correct
INDEX: 796 GROUND TRUTH: deceitful, RESPONSE: truthful - wrong
INDEX: 339 GROUND TRUTH: truthful, RESPONSE: deceitful - wrong
INDEX: 73 GROUND TRUTH: truthful, RESPONSE: deceitful - wrong


In [83]:
y_ground_truth = []  # for computing F1-score
y_predicted = []

for index in test_indices:
    infer_row = df.loc[index]  
    # print(f'Inferring the `class_outcome` for:\n{infer_row}')
    ground_truth = 'truthful' if infer_row['outcome_class'] == 't' else 'deceitful'
    # mask the `outcome_class` field since you want to predict it
    infer_row['outcome_class'] = ''

    # print(f'Original\n:{df.loc[index]}')
    # print(f'infer row\n: {infer_row}')

    prompt = construct_few_shot_prompt(random_truth_deceit_df, infer_row)
    prompt = ''.join(prompt)
    
    print(f'{prompt}')

    # response = get_chat_completion_with_backoff(
    #     prompt=prompt,
    #     model=MODEL,
    # )    
        
    # print(f'INDEX: {index} GROUND TRUTH: {ground_truth}, RESPONSE: {response} - {"wrong" if ground_truth != response else "correct"}')
    # y_ground_truth.append(ground_truth)
    # y_predicted.append(response)



You are tasked to judge whether the response to questions are truthful or deceitful.
You'll be presented with three pieces of information:
(1) The title of the activity.
(2) Response to question #1: “Please describe your activity. Be as specific as possible.”
(3) Response to question #2: “What information can you give us to reassure us that you are telling the truth”
Complete the response with either 'truthful' or 'deceitful'
Here are a few examples delimited by triple backticks:
```
Activity: 
On Sunday, my family and I are going to a local event called 'Festival of Heroes' at a nearby Roman Villa
Question #1: 
Please describe your activity. Be as specific as possible.
Response #1:
The festival runs over the whole weekend, but we only have tickets for the Sunday. It's an annual family festival where there are a lot of people dressed up as children's favourite characters. There's performances and meet and greets throughout the day.
Question #2: 
What information can you give us to rea

/tmp/ipykernel_9838/1993590657.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  infer_row['outcome_class'] = ''


### Compute metrics

In [80]:
from sklearn.metrics import f1_score
print('Weighted F1-score:', f1_score(y_ground_truth, y_predicted, average='weighted'))

Weighted F1-score: 0.41758241758241754
